In [1]:
import os
import queue
import tensorflow as tf
import cv2
import tensorflow_hub as hub
import numpy as np
from PIL import Image, ImageDraw
import csv
import sys
from Court_detection import CourtDetector
from MultiPose_estimation import MultiPoseEstimation
import pandas as pd
from Ball_tracker import BallTracker
from detect_actions import ActionDetector

2022-09-02 17:19:15.007198: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#Source video to test on 
input_video_path = "/home/JouiniAhmad/Desktop/douma/Badminton_Project/test.mp4"

In [3]:
ball_Tracker = BallTracker(input_video_path,"/home/JouiniAhmad/Desktop/douma/Badminton_Project/traknet_weights")
#predict method creates and saves csv label files
ball_Tracker.predict()

2022-09-02 17:19:18.994504: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-02 17:19:19.678348: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22269 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:5b:00.0, compute capability: 8.6
2022-09-02 17:19:22.953075: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400
2022-09-02 17:19:24.571692: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may 

1/1 [==============================] - 0s 32ms/step


In [4]:
#Source video to test on 
input_video_path = "/home/JouiniAhmad/Desktop/douma/Badminton_Project/ball_predicted/test_ball_predict.mp4"
#predicted ball trajectory csv file
input_csv_path = "/home/JouiniAhmad/Desktop/douma/Badminton_Project/ball_predicted/test_ball_predict.csv"

In [5]:
#load movenet model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [6]:
#call models
multipose = MultiPoseEstimation(2,'skeleton',movenet)
court_model = CourtDetector()
Action_model = ActionDetector("/home/JouiniAhmad/Desktop/douma/Badminton_Project/action_rec_weights.h5")

In [7]:
inputs = pd.DataFrame(columns=['Players_Positions', 'Players_Skeletons', 'Court_Center'])
#add 2 empty rows for first 2 frames 
inputs = inputs.append({'Players_Positions':[],'Players_Skeletons':[],'Court_Center': ()}, ignore_index=True)
inputs = inputs.append({'Players_Positions':[],'Players_Skeletons':[],'Court_Center': ()}, ignore_index=True)

ball_csv = pd.read_csv(input_csv_path)

/tmp/ipykernel_108182/2102608968.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inputs = inputs.append({'Players_Positions':[],'Players_Skeletons':[],'Court_Center': ()}, ignore_index=True)
/tmp/ipykernel_108182/2102608968.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inputs = inputs.append({'Players_Positions':[],'Players_Skeletons':[],'Court_Center': ()}, ignore_index=True)


In [8]:
with open(input_csv_path) as csvfile:
	readCSV = csv.reader(csvfile, delimiter=',')
	frames = []
	x, y = [], []
	list1 = []
	for row in readCSV:
		list1.append(row)
	for i in range(1 , len(list1)):
		frames += [int(list1[i][0])]
		x += [int(float(list1[i][2]))]
		y += [int(float(list1[i][3]))]

output_video_path = "./classifier_inputs/test_merged_Result.mp4"

q = queue.deque()
#we will save and plot the last 8 coordinates from current position
for i in range(0,8):
	q.appendleft(None)

In [9]:
def merge_frames(frame,img1):

    images =[]
    max_width = 0 # find the max width of all the images
    total_height = 0 # the total height of the images (vertical stacking)

    for name in [frame,img1]:
        # open all images and find their sizes
        images.append(name)
        if images[-1].shape[0] > total_height:
            total_height = images[-1].shape[0]
        max_width += images[-1].shape[1]    

    # create a new array with a size large enough to contain all the images
    final_image = np.zeros((total_height,max_width,3),dtype=np.uint8)

    return final_image,images

In [10]:
#get video fps&video size
currentFrame= 0
video = cv2.VideoCapture(input_video_path)
fps = int(video.get(cv2.CAP_PROP_FPS))
#output_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
#output_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#output_video = cv2.VideoWriter(output_video_path,fourcc, fps, (output_width,output_height))

video.set(1,currentFrame); 
ret, img1 = video.read()

#find_homography_matrix
imag1 = cv2.imread("/home/JouiniAhmad/Desktop/douma/Badminton_Project/data/images/court.jpg")
court = court_model.detect_court(img1)
pts_dst =np.array([(4,4),(267,4),(4,509),(267,509)])
H, status = cv2.findHomography(court[3], pts_dst)
imag2 = cv2.warpPerspective(img1, H,[imag1.shape[1],imag1.shape[0]])

final_image,images = merge_frames(img1,imag1)
output_width = final_image.shape[1]
output_height = final_image.shape[0]
output_video = cv2.VideoWriter(output_video_path,fourcc, fps, (output_width,output_height))

#write image to video
output_video.write(final_image)
currentFrame +=1
#input must be float type
img1 = img1.astype(np.float32)

#capture frame-by-frame
video.set(1,currentFrame)
ret, img = video.read()
final_image,images = merge_frames(img,imag1)
#write image to video
output_video.write(final_image)
currentFrame +=1
#input must be float type
img = img.astype(np.float32)

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [11]:
print("=========== Start Predicting ===========")
while(True):

	#capture frame-by-frame
	video.set(1,currentFrame); 
	ret, img = video.read()
		#if there dont have any frame in video, break
	if not ret: 
		break
	PIL_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
	PIL_image = Image.fromarray(PIL_image)

	if currentFrame in range(len(x)):
		if x[currentFrame] != 0 and y[currentFrame] != 0:
			q.appendleft([x[currentFrame],y[currentFrame]])
			q.pop()
		else:
			q.appendleft(None)
			q.pop()

	for i in range(0,8):
		if q[i] is not None:
			draw_x = q[i][0]
			draw_y = q[i][1]
			bbox =  (draw_x - 2, draw_y - 2, draw_x + 2, draw_y + 2)
			draw = ImageDraw.Draw(PIL_image)
			draw.ellipse(bbox, outline ='yellow')
			del draw
	opencvImage =  cv2.cvtColor(np.array(PIL_image), cv2.COLOR_RGB2BGR)
	img1 = opencvImage.copy()
	img1 = tf.image.resize_with_pad(tf.expand_dims(img1, axis=0), 384,640)
	input_img = tf.cast(img1, dtype=tf.int32)

	multipose.detect_player(input_img,opencvImage,court)

	Action_model.detect_stroke(multipose,opencvImage)

	#wrap positions to court reference with H transformation 
	if(len(multipose.positions) == 2):
		for p in multipose.positions:
			p = np.array([p], np.float32).reshape(-1,1,2).astype(np.float32)
			p =  np.round(np.squeeze(cv2.perspectiveTransform(p,H))).astype(int)
			cv2.circle(imag1,p,radius = 3, color=[0,0,255],thickness=-1)
	
	inputs = inputs.append({'Players_Positions':multipose.positions,'Players_Skeletons':multipose.skeletons,'Court_Center': multipose.center}, ignore_index=True)

	multipose.positions=[]
	multipose.skeletons=[]
	multipose.center=0

	final_image,images = merge_frames(opencvImage,imag1)
	current_x = 0 # keep track of where your current image was last placed in the y coordinate
	for image in images:
        # add an image to the final array and increment the y coordinate
		final_image[:image.shape[0],current_x:image.shape[1]+current_x,:] = image
		current_x += image.shape[1]

	#write image to output_video
	output_video.write(final_image)

	#next frame
	currentFrame += 1

inputs = pd.concat([inputs, ball_csv[['X','Y','Time']].copy()], axis=1)
inputs.rename(columns = {'X':'Ball_X', 'Y':'Ball_Y'}, inplace = True)
inputs.to_pickle("./classifier_inputs/test_inputs.pkl")
Action_model.save_actions("/home/JouiniAhmad/Desktop/douma/Badminton_Project/classifier_inputs/strokesA.csv",
"/home/JouiniAhmad/Desktop/douma/Badminton_Project/classifier_inputs/strokesB.csv",
input_csv_path,"/home/JouiniAhmad/Desktop/douma/Badminton_Project/classifier_inputs/A_actions.csv",
"/home/JouiniAhmad/Desktop/douma/Badminton_Project/classifier_inputs/B_actions.csv")

video.release()
output_video.release()
print("=========== Finish Predicting ===========")

=========== Start Predicting ===========


/home/JouiniAhmad/Desktop/douma/Badminton_Project/MultiPose_estimation.py:95: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "detect_player" failed type inference due to: non-precise type pyobject
During: typing of argument at /home/JouiniAhmad/Desktop/douma/Badminton_Project/MultiPose_estimation.py (98)

File "MultiPose_estimation.py", line 98:
    def detect_player(self,input_img,frame,court):
        <source elided>

        self.center = court[1] 
        ^

  @jit
/home/JouiniAhmad/Desktop/douma/Badminton_Project/MultiPose_estimation.py:95: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "detect_player" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "MultiPose_estimation.py", line 120:
    def detect_player(self,input_img,frame,court):
        <source elided>
            #range is same passed to find_player
     

1/1 [==============================] - 0s 24ms/step


/home/JouiniAhmad/.local/lib/python3.8/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "draw_keypoints" was compiled in object mode without forceobj=True.

File "MultiPose_estimation.py", line 48:
    @jit 
    def draw_keypoints(self,frame, keypoints):
    ^

  warnings.warn(errors.NumbaWarning(warn_msg,
/home/JouiniAhmad/.local/lib/python3.8/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "MultiPose_estimation.py", line 48:
    @jit 
    def draw_keypoints(self,frame, keypoints):
    ^

  warnings.warn(errors.NumbaDeprecationWarning(msg,
2022-09-02 17:20:37.033230: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be u

1/1 [==============================] - 0s 25ms/step


...

1/1 [==============================] - 0s 52ms/step


/home/JouiniAhmad/Desktop/douma/Badminton_Project/detect_actions.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.strokesA = self.strokesA.append({'Player_A':'___'}, ignore_index=True)
/home/JouiniAhmad/Desktop/douma/Badminton_Project/detect_actions.py:164: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.strokesB = self.strokesB.append({'Player_B':'___'}, ignore_index=True)
/tmp/ipykernel_108182/3855559223.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inputs = inputs.append({'Players_Positions':multipose.positions,'Players_Skeletons':multipose.skeletons,'Court_Center': multipose.center}, ignore_index=True)


1/1 [==============================] - 0s 25ms/step


/home/JouiniAhmad/Desktop/douma/Badminton_Project/detect_actions.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.strokesA = self.strokesA.append({'Player_A':'___'}, ignore_index=True)
/home/JouiniAhmad/Desktop/douma/Badminton_Project/detect_actions.py:164: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.strokesB = self.strokesB.append({'Player_B':'___'}, ignore_index=True)
/tmp/ipykernel_108182/3855559223.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inputs = inputs.append({'Players_Positions':multipose.positions,'Players_Skeletons':multipose.skeletons,'Court_Center': multipose.center}, ignore_index=True)


1/1 [==============================] - 0s 26ms/step
